In [1]:
import os
import pathlib
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import Adam
from PIL import ImageFile
import glob
import numpy as np

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
data_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [4]:
train_path='E:\\ml\\train'
test_path='E:\\ml\\test'

train_loader= DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=data_transforms),
    batch_size=32, shuffle=True
)
test_loader= DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=data_transforms),
    batch_size=32, shuffle=True
)

In [5]:
root = pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['Albatross', 'Bulbul', 'Dove', 'Duck', 'Eagle', 'Hummingbird', 'Kingfisher', 'Macaw', 'Ostrich', 'Owl', 'Peacock', 'Penguin', 'Woodpecker']


In [8]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=13):
        super(ConvNet, self).__init__()
        
        #((w-f+2P)/s) +1
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        # Fully Connected Layers
        self.fc1 = nn.Linear(in_features=128 * 28 * 28, out_features=512)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=256, out_features=num_classes)

    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool1(output)

        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.pool2(output)

        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.pool3(output)

        output = output.view(-1, 128 * 28 * 28)

        output = self.fc1(output)
        output = self.relu4(output)
        output = self.fc2(output)
        output = self.relu5(output)
        output = self.fc3(output)

        return output

In [9]:
model= ConvNet(num_classes=13).to(device)

In [10]:
optimizer=Adam(model.parameters(), lr=0.001, weight_decay = 0.0001)
loss_function=nn.CrossEntropyLoss()

In [11]:
num_epochs=10

In [12]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [13]:
print(train_count, test_count)

3970 125


In [14]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
best_accuracy=0.0

for epoch in range (num_epochs):
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate (train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate (test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: ' +str(epoch)+ ' Train Loss: ' +str(train_loss)+ ' Train Accuracy: ' +str(train_accuracy)+ ' Test Accuracy: ' +str(test_accuracy))
    
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(3.8134) Train Accuracy: 0.28463476070528965 Test Accuracy: 0.392
Epoch: 1 Train Loss: tensor(1.8208) Train Accuracy: 0.38413098236775817 Test Accuracy: 0.384
Epoch: 2 Train Loss: tensor(1.7244) Train Accuracy: 0.4329974811083123 Test Accuracy: 0.472
Epoch: 3 Train Loss: tensor(1.6466) Train Accuracy: 0.46574307304785895 Test Accuracy: 0.488
Epoch: 4 Train Loss: tensor(1.4900) Train Accuracy: 0.5093198992443325 Test Accuracy: 0.576
Epoch: 5 Train Loss: tensor(1.4102) Train Accuracy: 0.5438287153652392 Test Accuracy: 0.512
Epoch: 6 Train Loss: tensor(1.3890) Train Accuracy: 0.5546599496221662 Test Accuracy: 0.704
Epoch: 7 Train Loss: tensor(1.2803) Train Accuracy: 0.5874055415617129 Test Accuracy: 0.672
Epoch: 8 Train Loss: tensor(1.2371) Train Accuracy: 0.5921914357682619 Test Accuracy: 0.584
Epoch: 9 Train Loss: tensor(1.1647) Train Accuracy: 0.6244332493702771 Test Accuracy: 0.64
